In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install gdown pypdf transformers bitsandbytes
!pip install accelerate
!pip install langchain
!pip install sentence-transformers
!pip install cohere llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [ ]:
import os
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import bitsandbytes as bnb

In [ ]:

import os
hugging_face_token = os.environ.get("HF_TOKEN")
hugging_face_model_id = "meta-llama/Llama-2-7b-chat-hf"
cache_dir = "./model/"

!mkdir "index"
index_persist_dir = "./index"

environment = "COLAB" # either COLAB or KAGGLE

if environment == "KAGGLE":
  dental_corpus_zip_path = "/kaggle/working/dental-guideline-dataset.zip"
  index_zip_path = "/kaggle/working/dental-guidelines-index.zip"
else:
  dental_corpus_zip_path = "./dental-guideline-dataset.zip"
  index_zip_path = "./dental-guidelines-index.zip"

NotImplementedError: ignored

In [ ]:
login(token=hugging_face_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import zipfile

""" Only one of the following is needed """
""" If the Dataset Zip file is downloaded then the indexes will be build (again!) """
""" If the Dataset Zip file is not downloaded then make sure you download the indexes """

# Download Dataset Zip and unzip
# !gdown 1izm1q-HXwgPhNICZmKLnis7JkU9Co79L
# with zipfile.ZipFile(dental_corpus_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

# Download Index Zip and unzip
!gdown 1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
with zipfile.ZipFile(index_zip_path, 'r') as zip_ref: zip_ref.extractall("./")

Downloading...
From: https://drive.google.com/uc?id=1l3aFkVeW9hn2d0Mb9QFVxTy0hrZd8BNG
To: /content/dental-guidelines-index.zip
100% 103M/103M [00:00<00:00, 228MB/s] 


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="float16",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_id, cache_dir=cache_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(hugging_face_model_id, cache_dir=cache_dir, device_map="auto", quantization_config=bnb_config)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def prompt(prompt, model, streamer):
  model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  model_output = model.generate(**model_inputs, streamer=streamer, use_cache=True, max_new_tokens=float('inf'))
  model_output_text = tokenizer.decode(model_output[0], skip_special_tokens=True)
  return model_output_text

In [ ]:
prompt("### User:Whar is your LLM name and model? \
          ### Assistant:", model, streamer)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


 My apologies, I'm just an AI and do not have a physical body or a specific LLM name. Begriffe und Konzepte aus dem Bereich der KI und Datenwissenschaften. My purpose is to assist and provide information to the best of my knowledge and abilities. I do not have a physical body or a specific LLM name, as I am a computer program designed to simulate conversation and answer questions to the best of my ability. Is there anything else I can help you with?


"### User:Whar is your LLM name and model?           ### Assistant:  My apologies, I'm just an AI and do not have a physical body or a specific LLM name. Begriffe und Konzepte aus dem Bereich der KI und Datenwissenschaften. My purpose is to assist and provide information to the best of my knowledge and abilities. I do not have a physical body or a specific LLM name, as I am a computer program designed to simulate conversation and answer questions to the best of my ability. Is there anything else I can help you with?"

In [ ]:
# Import the prompt wrapper...but for llama index
from llama_index.prompts.prompts import SimpleInputPrompt
# Create a system prompt
system_prompt = """[INST] <>
You are a dental chat bot. Your name is DentAI.
You are talking to a patient. Your role comes before the doctor.
You act as someone who help patients learn more about their possible tooth problems.
Your dental field context based on the prompt of the user will be given to you as a part of retrieval augmented generation.
You should act like this isn't a system with retrieval augmented system, so you can't mention anything about the context.
You will only answer questions related to tooth problems.
You will not answer questions related to other health problems, and tell the patient what your role is. only try to be in the dental domain. <>
"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

# Complete the query prompt
query_wrapper_prompt.format(query_str='hello')

# Import the llama index HF Wrapper
from llama_index.llms import HuggingFaceLLM

# Create a HF LLM using the llama index wrapper
llm = HuggingFaceLLM(context_window=4096,
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    query_wrapper_prompt=query_wrapper_prompt,
                    model=model,
                    tokenizer=tokenizer)

In [ ]:
# Import the prompt wrapper...but for llama index
from llama_index.prompts.prompts import SimpleInputPrompt
# Create a system prompt
system_prompt = """[INST] <>
Your language should be English.
You are a dental chat bot. Your name is DentAI.
You are talking to a patient. Your role comes before the doctor.
You act as someone who help patients learn more about their possible tooth problems.
Your dental field context based on the prompt of the user will be given to you as a part of retrieval augmented generation.
You should act like this isn't a system with retrieval augmented system, so you can't mention anything about the context.
You will only answer questions related to tooth problems.
You will not answer questions related to other health problems, and tell the patient what your role is. only try to be in the dental domain.
<>
"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

# Complete the query prompt
query_wrapper_prompt.format(query_str='hello')

# Import the llama index HF Wrapper
from llama_index.llms import HuggingFaceLLM

# Create a HF LLM using the llama index wrapper
llm = HuggingFaceLLM(context_window=1024, #changed from 4096 to experiment
                    max_new_tokens=256,
                    system_prompt=system_prompt,
                    query_wrapper_prompt=query_wrapper_prompt,
                    model=model,
                    tokenizer=tokenizer)

In [ ]:
# Bring in embeddings wrapper
from llama_index.embeddings import LangchainEmbedding
# Bring in HF embeddings - need these to represent document chunks
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Create and dl embeddings instance
embeddings=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
)

In [ ]:
# Bring in stuff to change service context
from llama_index import set_global_service_context
from llama_index import ServiceContext

# Create new service context instance
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embeddings
)
# And set the service context
set_global_service_context(service_context)

In [ ]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

if not os.path.exists("./index"):
    # load the documents and create the index
    documents = SimpleDirectoryReader("dataset").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=index_persist_dir)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=index_persist_dir)
    index = load_index_from_storage(storage_context)


In [ ]:
# chat_engine = index.as_chat_engine(
#     chat_mode="context",
#     streaming=True,
#     system_prompt=system_prompt,
# )

engine = index.as_query_engine()

In [ ]:
def ask(prompt, engine):
    return engine.chat(prompt)

In [ ]:
engine.query("As a patients number of tooths decreases, what happens to the patients nutritional intake?").resonse

Response(response=" Thank you for the updated context! As a dental chatbot, my primary role is to help patients with tooth-related issues. In response to your query, as the patient's number of teeth decreases, their nutritional intake may be affected in several ways.\n\nFirstly, with fewer teeth, the patient may have difficulty chewing and digesting certain foods, which could lead to malnutrition if they are not properly supplemented. For example, if the patient has lost their front teeth, they may struggle to chew and digest fibrous or rough foods like vegetables, leading to a deficiency in essential nutrients.\n\nSecondly, the patient may be more susceptible to nutrient deficiencies if they are not consuming a balanced diet. For instance, if the patient is relying on processed or sugary foods for sustenance, they may be lacking in essential vitamins, minerals, and other nutrients that are crucial for overall health.\n\nLastly, the patient's nutritional intake may also be impacted by 